# Work with a MongoDB docker image

## Docker commands

<mark>**Change the kernel to powershell, before you run the code below.**</mark>

### List all Images in the docker repository

In [1]:
docker images -a

REPOSITORY   TAG       IMAGE ID       CREATED        SIZE
mongo        latest    f03be0dc25f8   28 hours ago   448MB


### Check status of running Images/Containers

In [2]:
docker ps

CONTAINER ID   IMAGE     COMMAND                  CREATED         STATUS         PORTS                   NAMES
fa5f7fc7481c   mongo     "docker-entrypoint.s…"   3 minutes ago   Up 3 minutes   0.0.0.0:80->27017/tcp   mongodb


## Create a MongoDB container

Download latest version of mongDB

In [ ]:
docker pull mongo

Configure the data-folder on the local-system instead of the docker image

In [ ]:
mkdir -p /mongodata

## Start the MongoDB container

Start the docker image with name **<mark>mongodb</mark>** and mapp the port to 80

In [ ]:
docker run -p 80:27017 -it -v mongodata:/data/db --name mongodb -d mongo
docker ps

### Inspect the container

Run docker inspect **CONTAINR ID**

In [ ]:
$inspected_image = docker inspect fa5f7fc7481c
$object = $inspected_image | ConvertFrom-Json
$object | Select Name, @{l="IPAddress";e={$object.NetworkSettings.IPAddress}}, @{l="Ports";e={($object.NetworkSettings.Ports)}}

### Show the logs of the container

In [ ]:
docker logs mongodb

### Stop and restart MongoDB container

In [ ]:
docker stop mongodb
docker ps -a

In [ ]:
docker start mongodb
docker ps -a

Remove docker container

In [ ]:
docker rm mongodb
docker ps -a

## Python

<mark>**Please change the kernel to python, before you run the code below.**</mark>

### Insert one document

Connect to the docker mongodb and insert one document in the database JupyterNB, collection PoweredOffVMs

In [ ]:
def add_document(connectionstring, mongodatabase, collection, document):
    '''Connect to MongoDB and add a document'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    mongo_col.insert_one(document)
    for doc in mongo_col.find():
        print(doc)

    mongo_client.close()

connectionstring = "mongodb://0.0.0.0:80"
add_document(connectionstring, 'JupyterNB', 'PoweredOffVMs', { "PSComputerName": "xsc0000991.child.domain.com", "PowerState": "Off" })

### List all documents

Connect to the docker mongodb and list all documents from a collection

In [1]:
def get_documents(connectionstring, database, collection):
    '''Connect to MongoDB and return data'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[database]
    mongo_col    = mongo_db[collection]
    mongo_client.close()
    return mongo_col.find()

import pandas as pd
table = []
connectionstring = "mongodb://0.0.0.0:80"
result = get_documents(connectionstring, 'JupyterNB', 'PoweredOffVMs')
for doc in result:
    #print(doc)
    thisdict = { 
        'PSComputerName' : doc['PSComputerName'],
        'PowerState'     : doc['PowerState'],
    }
    table.append(thisdict)

df = pd.DataFrame(table)
df.head()

,PSComputerName,PowerState
0,xsc0000989.child.domain.com,Off
1,xsc0000990.child.domain.com,Off
2,xsc0000991.child.domain.com,Off


### Delete all documents

Connect to the docker mongodb and delete all documents from collection

In [ ]:
def delete_collection(connectionstring, mongodatabase, collection):
    '''Connect to MongoDB and add a document'''
    import pymongo
    mongo_client = pymongo.MongoClient(connectionstring)
    mongo_db     = mongo_client[mongodatabase]
    mongo_col    = mongo_db[collection]
    mongo_col.delete_many({})
    for doc in mongo_col.find():
        print(doc)

    mongo_client.close()

connectionstring = "mongodb://0.0.0.0:80"
delete_collection(connectionstring,'JupyterNB', 'PoweredOffVMs')

## Interactive docker terminal

Start an interactive docker terminal to manage MongoDB. Copy the code into a shell and press enter:

```
docker exec -it mongodb bash
```

## Mongo shell

[Mongo Shell Quick Reference](https://docs.mongodb.com/manual/reference/mongo-shell/)

```
mongo
```
```
show dbs
show collections
show users
```